<div style="text-align: center;">

## **Local Chatbot with Postgres Integration**

</div>

<div style="text-align: justify;">

This project showcases a local chatbot implementation that uses a PostgreSQL database to store and retrieve chat conversation history. The diagram below illustrates the chatbot's high-level workflow.

In this system, the user provides a prompt to the chatbot, which is then saved to its memory via the PostgreSQL database. LangChain retrieves both the user prompt and the stored memory from the database. These inputs are processed by the OpenAI chat model, which determines whether to generate a response based solely on the user prompt or by incorporating the stored memory. OpenAI is chosen due to harwdware's resource constraint on running opensource LLMs. Finally, the model's output becomes the chatbot's response, which is also saved back into the database.

</div>

<div style="text-align: center;">
    <figure>
        <img src="images/block_diagram.png" alt="Alt text" width="1000"/>
        <figcaption style="margin-top: 10px; font-style: Normal;">
            Figure 1: High-level workflow of the local chatbot.
        </figcaption>
    </figure>
</div>


### Setting Up PostgreSQL Database with Chat History Table

To set up the PostgreSQL database for storing chat history, follow the steps below. This setup uses a `Dockerfile`, `docker-compose.yaml`, and an `init.sql` script to automate the database initialization.

#### Dockerfile
```dockerfile
FROM postgres:latest

COPY init.sql /docker-entrypoint-initdb.d/

EXPOSE 5432
```

- **Base Image**: Uses the latest official PostgreSQL image.
- **Initialization Script**: The `init.sql` file is copied into the container's initialization directory. PostgreSQL automatically runs this script during the container startup.
- **Port Exposure**: Exposes port `5432` for database connections.

#### docker-compose.yaml
```yaml
services:
  postgres:
    build:
      context: .
    container_name: chatbot_postgres
    ports:
      - "5432:5432"
    volumes:
      - /var/lib/postgresql/data
    environment:
      POSTGRES_USER: ${POSTGRES_USER}
      POSTGRES_PASSWORD: ${POSTGRES_PASSWORD}
      POSTGRES_DB: ${POSTGRES_DB}
    env_file:
      - .env
```

- **Build Context**: Builds the PostgreSQL image using the `Dockerfile` in the current directory.
- **Container Name**: Names the container `chatbot_postgres` for easier identification.
- **Ports**: Maps port `5432` of the container to the host machine.
- **Volumes**: Temporary persists PostgreSQL data in `/var/lib/postgresql/data`.
- **Environment Variables**: Reads database credentials from the `.env` file for secure configuration.

#### init.sql
```sql
CREATE TABLE IF NOT EXISTS chat_history (
    id SERIAL PRIMARY KEY,
    session_id UUID NOT NULL,
    role TEXT NOT NULL,
    content TEXT NOT NULL,
    timestamp TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);
```

**Table Creation**: Defines the `chat_history` table with the following columns:
  - `id`: A unique identifier for each entry.
  - `session_id`: A UUID to group messages by session.
  - `role`: Specifies whether the message is from the user or the assistant.
  - `content`: The message text.
  - `timestamp`: Automatically records the time of entry.

After setting up the three files, database can be created by running `docker-compose up --build` command in any device's terminal.

### Code Implementation

Assuming that `requirements.txt` is already installed, we can now import the necessary packages. I will be discussing later the importance of each packages.

In [2]:
import streamlit as st
import uuid
import os
import psycopg
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_postgres import PostgresChatMessageHistory

#### Load and initialize the model

Here, `gpt-3.5-turbo` was used since it is good in conversational tasks and is cost-effective compared to GPT-4 models.

In [3]:
# Load OpenAI API Key
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

# Initialize the model
model = ChatOpenAI(model = "gpt-3.5-turbo")

#### Initialize the prompt template

The next code initializes a prompt template for a conversational AI model, where a user’s question is inserted into the human_template. The `MessagesPlaceholder(variable_name="history")` creates a placeholder for including chat history in the prompt. This allows the model to incorporate past messages for generating contextually relevant responses.

In [4]:
human_template = f"{{question}}"
prompt_template = ChatPromptTemplate.from_messages(
    [
        MessagesPlaceholder(variable_name="history"),
        ("human", human_template),
    ]
)

#### Chain the prompt and model. Define table name

Here, the code chains the prompt template with the model This creates a pipeline where the model generates responses based on the structured input. It also defines the table name `chat_history` for storing chat history in a Postgres database. The chat history will be stored and retrieved from this table for each user session.

In [7]:
# Chain the prompt and model
chain = prompt_template | model

# Define table name for PostgreSQL
table_name = "chat_history"

#### Function to retrieve chat history

This code defines a function `get_by_session_id` that retrieves chat history from a PostgreSQL database based on a given session ID. The function returns a PostgresChatMessageHistory object, which loads the chat history for the specified session from the `chat_history` table.

`BaseChatMessageHistory` is an abstract class that provides a common interface for managing chat message history across different storage systems. In this code, `PostgresChatMessageHistory` inherits from it which offers a Postgres-specific implementation to retrieve and manage chat history for a given session ID.

In [8]:
def get_by_session_id(session_id: str) -> BaseChatMessageHistory:
    sync_connection = psycopg.connect(
        dbname=os.getenv("POSTGRES_DB"),
        user=os.getenv("POSTGRES_USER"),
        password=os.getenv("POSTGRES_PASSWORD"),
        host="localhost",
        port="5432",
    )
    return PostgresChatMessageHistory(
        table_name, session_id, sync_connection=sync_connection
    )

#### Chain the prompt, model, and history retriever

The next code enables the model to generate responses considering both the current question, and the conversation history.

In [9]:
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_by_session_id,
    input_messages_key="question",
    history_messages_key="history",
)

#### Function to delete chat history

This code defines a function `delete_chat_history` that removes chat history from a PostgreSQL database for a given session ID. It establishes a connection to the database, executes a `DELETE` SQL query to remove the records from the `chat_history` table, and commits the changes.

This function will be used whenever the user intends to create a new chat conversation with the chatbot.

In [10]:
def delete_chat_history(session_id: str):
    with psycopg.connect(
        dbname=os.getenv("POSTGRES_DB"),
        user=os.getenv("POSTGRES_USER"),
        password=os.getenv("POSTGRES_PASSWORD"),
        host="localhost",
        port="5432",
    ) as conn:
        with conn.cursor() as cur:
            cur.execute(
                f"DELETE FROM {table_name} WHERE session_id = %s", (session_id,)
            )
            conn.commit()

#### Initialize Streamlit session variables

Here, the code make sure that a new, unique session ID through `uuid` library will be created whenever no session ID exists. It also initializes the `chat_history` as an empty list if it does not exist. 

In [12]:
if "session_id" not in st.session_state:
    st.session_state.session_id = str(uuid.uuid4())  # Generate unique session ID for a user

if "chat_history" not in st.session_state:
    st.session_state.chat_history = []  # Initialize chat history

2025-01-08 20:05:07.599 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-08 20:05:07.602 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-08 20:05:07.603 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-08 20:05:07.603 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-08 20:05:07.603 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-08 20:05:07.603 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


#### Streamlit User Interface

The code will mimic how the chatbot works. It will display first the title `Local Chatbot using ChatOpenAI`, then the conversations will be also displayed in the interface. A streamlit's  chat_input` was also set to let the users type their queries.

In [14]:
# Title of the website
st.title("Local Chatbot using ChatOpenAI")

# Display chat history
for message in st.session_state.chat_history:
    if message["role"] == "user":
        st.chat_message("user").markdown(message["content"])
    else:
        st.chat_message("assistant").markdown(message["content"])

# Request user to input prompt
user_question = st.chat_input("Message Me: ")

2025-01-08 20:05:50.798 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-08 20:05:50.803 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-08 20:05:50.804 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-08 20:05:50.805 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-08 20:05:50.807 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-08 20:05:50.810 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-08 20:05:50.813 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-08 20:05:50.816 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

#### One Conversation Workflow

When the user submits a question, it is displayed in the chat, and the model's response is generated using `chain_with_history`, which considers the chat history for context. The response from the model is then shown in the chat, and both the user’s question and the assistant’s response are saved to the chat_history for the current session.

In [15]:
# User submits the question
if user_question:
    # Show user message in the chat
    st.chat_message("user").markdown(user_question)

    # Get the chatbot's response
    result = chain_with_history.invoke(
        {"question": user_question},
        config={"configurable": {"session_id": st.session_state.session_id}},
    )
    
    # Show chatbot's response in the chat
    st.chat_message("assistant").markdown(result.content)
    
    # Save the conversation in chat history
    st.session_state.chat_history.append({"role": "user", "content": user_question})
    st.session_state.chat_history.append({"role": "assistant", "content": result.content})

#### For New Conversation

This deletes the current chat history from the Postgres database, generates a new unique session ID, and clears the chat_history in the session state. After these changes, it reruns the Streamlit app which resets the chat interface for the new conversation.

In [16]:
# When new conversation is initiated
if st.button("Start New Conversation"):
    # Delete chat history from the PostgreSQL database
    delete_chat_history(st.session_state.session_id)
    
    # Generate a new session ID
    st.session_state.session_id = str(uuid.uuid4())
    
    # Clear chat history in website's session state
    st.session_state.chat_history = []
    
    # Rerun the app
    st.rerun()

2025-01-08 20:09:48.291 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-08 20:09:48.292 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-08 20:09:48.293 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-08 20:09:48.293 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-08 20:09:48.293 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


#### END

This is the end. Since this is a Jupyter Notebook, I cannot directly run the code. However, you can look at the link below for the video demo of the created chatbot.

**Demo**: https://drive.google.com/file/d/1PaKOBu-HsbcBWVB8cfexGPsqWhKz9t4c/view?usp=sharing


**Additional Note**
- You can run the code using `streamlit run app.py` command. The file `app.py` contains the same code here in the documentation.
- GitHub Repository: https://github.com/Rob-Christian/My-Local-Chatbot
- I did not use opensource LLM and Llama-Cpp-Python due to hardware resource constraint. A single prompt inference of the chatbot takes more than a minute for 4-bit quantized version of Llama-7b. The inference time is also increasing as the conversation continues due to added memory input to the model.
